# Crawling

In [1]:
import os

def mkdir(path):
    # Membuat folder jika belum ada
    if not os.path.exists(path):
        os.makedirs(path)

def mkdir_r(path):
    path = os.path.normpath(path)
    paths = path.split('/')
    output_path = '/'.join(paths)

    for i in range(1, len(paths)+1):
        mkdir('/'.join(paths[0:i]))
    


In [2]:
from bs4 import BeautifulSoup
import datetime
import requests
import os
import re
import subprocess

# Fungsi crawling portal viva.co.id
# date format YYYY-MM-DD
def viva_crawling(kanal="all", sub_kanal="all", start_date=datetime.datetime.today(), end_date=datetime.datetime.today() + datetime.timedelta(days=1)):
    kanal = kanal.lower()
    sub_kanal = sub_kanal.lower()
    
    # URL dasar yang akan diunduh
    index_url = 'https://www.viva.co.id/indeks'

    current_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")

    if os.path.exists(f'urls/list_{kanal}_{sub_kanal}.txt'):
        os.remove(f'urls/list_{kanal}_{sub_kanal}.txt')

    count = 0
    # Loop melalui tanggal dari tanggal awal ke tanggal akhir
    while current_date.strftime("%Y-%m-%d") != end_date:
        # Memisahkan tanggal menjadi tahun, bulan, dan tanggal
        year = current_date.strftime("%Y")
        month = current_date.strftime("%m")
        day = current_date.strftime("%d")
        print(f'{year}-{month}-{day}')

        url = f'{index_url}/{kanal}/{sub_kanal}/{year}/{month}/{day}'
        try:
                req = requests.get(url)
                bs = BeautifulSoup(req.text, 'html.parser')

                links = bs.find_all('a', attrs={'class': 'article-list-thumb-link flex_ori'})

                for link in links:
                    href = link['href']
                    mkdir('urls')
                    # Memeriksa apakah URL dimulai dengan "https://www.viva.co.id/{kanal}/{sub_kanal}"
                    with open(f'urls/list_{kanal}_{sub_kanal}.txt', 'a') as file:
                        if href.startswith(f'https://www.viva.co.id/{kanal}/'):
                            # print(f'href="{href}')

                            # Simpan href di dalam file list-edukasi.txt
                            file.write(href + '\n')

                            count += 1

                            mkdir_r(f'./htmls/{kanal}/{sub_kanal}/')
                            # Mengunduh dengan curl
                            subprocess.call(['curl', '-o', f'./htmls/{kanal}/{sub_kanal}/{count}.html', href])

                # Menambahkan satu hari ke tanggal saat ini
                current_date += datetime.timedelta(days=1)
        except requests.exceptions.RequestException as e:
            print(f"Terjadi kesalahan dalam mengakses {html}: {e}")    
    
        print("Jumlah Halaman : ", count)


In [3]:
# crawling portal berita viva 

viva_crawling("edukasi", "all", "2023-05-01", "2023-05-02")
viva_crawling("sport", "all", "2023-05-01", "2023-05-02")
# viva_crawling("bola", "all", "2023-05-01", "2023-05-02")

2023-05-01


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  157k    0  157k    0     0   495k      0 --:--:-- --:--:-- --:--:--  498k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  164k    0  164k    0     0   496k      0 --:--:-- --:--:-- --:--:--  497k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  149k    0  149k    0     0   646k      0 --:--:-- --:--:-- --:--:--  648k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  159k    0  159k    0     0   429k      0 --:--:-- --:--:-- --:--:--  431k
  % Total    % Received % Xferd  Average Speed   Tim

Jumlah Halaman :  25
2023-05-01


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  147k    0  147k    0     0   423k      0 --:--:-- --:--:-- --:--:--  425k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  148k    0  148k    0     0   419k      0 --:--:-- --:--:-- --:--:--  421k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  147k    0  147k    0     0   434k      0 --:--:-- --:--:-- --:--:--  435k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  152k    0  152k    0     0   492k      0 --:--:-- --:--:-- --:--:--  493k
  % Total    % Received % Xferd  Average Speed   Tim

Jumlah Halaman :  25


100  148k    0  148k    0     0   377k      0 --:--:-- --:--:-- --:--:--  377k


In [4]:
# convert html to txt
import os


# Fungsi convert html to txt
def htmls_to_txts(source):
    source = os.path.normpath(source)
    paths = source.split('/')
    paths[0] = 'txts'
    output_path = '/'.join(paths)

    mkdir_r(output_path)

    # Membaca dan mendapatkan semua file dalam folder
    files = [f for f in os.listdir(source) if os.path.isfile(os.path.join(source, f))]

    i = 1
    for file in files:
        file_path = os.path.join(source, file)

        # Membagi string berdasarkan tanda titik (.)
        file_parts = file.split('.')

        # Mengambil bagian terakhir, yang seharusnya menjadi ekstensi
        file_extension = file_parts[-1]

        file_txt = file.replace('.' + file_extension, '.txt')

        output_file = os.path.join(output_path, file_txt)
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                soup = BeautifulSoup(content, 'html.parser')
                main_content_titles = soup.find_all(class_='main-content-title')
                main_content_details = soup.find(class_='main-content-detail')
                with open(output_file, 'w', encoding='utf-8') as output:
                    if main_content_titles:
                        for title in main_content_titles:
                            output.write(title.text + '\n')
                    if main_content_details:
                        paragraphs = main_content_details.find_all('p')
                        for paragraph in paragraphs:
                            isi = paragraph.get_text()
                            output.write(isi + '\n')

                print(f"Processed {file_path} and wrote to {output_file}")
        except FileNotFoundError as e:
            print(e)
        except Exception as e:
            print(f"An error occurred while processing {file_path}: {e}")



In [5]:
# Panggil fungsi html to txt
htmls_to_txts("./htmls/edukasi/all/")
htmls_to_txts("./htmls/sport/all/")
# htmls_to_txts("./htmls/bola/all/")
print('Converted to txt Done!')

Processed htmls/edukasi/all/1.html and wrote to txts/edukasi/all/1.txt
Processed htmls/edukasi/all/2.html and wrote to txts/edukasi/all/2.txt
Processed htmls/edukasi/all/3.html and wrote to txts/edukasi/all/3.txt
Processed htmls/edukasi/all/4.html and wrote to txts/edukasi/all/4.txt
Processed htmls/edukasi/all/5.html and wrote to txts/edukasi/all/5.txt
Processed htmls/edukasi/all/6.html and wrote to txts/edukasi/all/6.txt
Processed htmls/edukasi/all/7.html and wrote to txts/edukasi/all/7.txt
Processed htmls/edukasi/all/8.html and wrote to txts/edukasi/all/8.txt
Processed htmls/edukasi/all/9.html and wrote to txts/edukasi/all/9.txt
Processed htmls/edukasi/all/10.html and wrote to txts/edukasi/all/10.txt
Processed htmls/edukasi/all/11.html and wrote to txts/edukasi/all/11.txt
Processed htmls/edukasi/all/12.html and wrote to txts/edukasi/all/12.txt
Processed htmls/edukasi/all/13.html and wrote to txts/edukasi/all/13.txt
Processed htmls/edukasi/all/14.html and wrote to txts/edukasi/all/14.